# Assignment 1

- Pick an occupation and filter data accordingly. You must all pick different occupations / occupation individually.
- Occupation codes are here: https://osf.io/57n9q/
- You may merge occupations as you see fit (ie all tax/insurance specialists, etc).

Build four predictive models using linear regression for earnings per hour.
1. Models: the target variable is earnings per hour, all others would be predictors

In [1]:
# Importing the required libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [3]:
# Reading the data
data = pd.read_csv("https://osf.io/g8p9j/")

# Checking if the data was correctly loaded
data.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2
